### Prepare space in /flash

In [1]:
import os, subprocess

In [2]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "3D")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Main

In [3]:
import pandas as pd
import numpy as np
import os, h5py
from probemapper.plot import merge_data_oct
from probemapper.plot import threeD

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/scratch/runtime-tomoyuki-mano1'


In [4]:
lut = pd.read_csv("/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/region_LUT.csv")
lut["region_name2"] = ["VL", "Others", "sFL", "iFL", "Buc", "Subfr", "aBL", "dBL", "Subv", "Prec"]
display(lut)

,ID,region_name,region_color,region_name2
0,1,Vertical Lobe,"128,174,128",VL
1,3,Others,"216,100,79",Others
2,4,Median superior frontal lobe + Lateral superio...,"216,100,79",sFL
3,5,Median inferior frontal lobe + Lateral inferio...,"255,147,114",iFL
4,6,Superior buccal lobe\n+ Posterior buccal lobe,"244,128,26",Buc
5,7,Subfrontal lobe,"251,202,0",Subfr
6,8,Anterior basal lobe,"89,114,255",aBL
7,9,Anterior dorsal basal lobe + Posterior dorsal ...,"82,145,163",dBL
8,11,Subvertical Lobe,"24,151,54",Subv
9,12,Precomissural,"255,170,255",Prec


In [5]:
data = [
    {
        "name": "L-J30",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220526_P189_laqueus_DiI_L-J30/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 0,
    },
    {
        "name": "L-F11",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220526_P190_laqueus_DiI_L-F11/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 1,
    },
    {
        "name": "L-M4",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220321_P177_laqueus_DiI_3-8_CUBIC/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 2,
    },
    {
        "name": "L-M13",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220529_P191_laqueus_DiI_LM13/analysis/indexing/channel_LUT.csv",
        "offset": 10,
        "hdf5_column": 3,
    },
    {
        "name": "L-M27",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220517_P185_laqueus_DiI_L-M27/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 4,
    },
    {
        "name": "L-A1",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220613_P200_laqueus_DiI_L-A1/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 5,
    },
    {
        "name": "L-A2",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220518_P186_laqueus_DiI_L-A2/analysis/indexing/channel_LUT.csv",
        "offset": 10,
        "hdf5_column": 6,
    },
    {
        "name": "L-A11",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220614_P201_laqueus_DiI_L-A11/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 7,
    },
    {
        "name": "L-A29",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220615_P202_laqueus_DiI_L-A29/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 8,
    }
]

In [6]:
hdf_info = [
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/asBoutOverChans150.h5",
        "key": "meanL",
        "name": "AS_low",
        "description": "Envelop intensity (a.u.) in 0.1-10 Hz band during AS"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/asBoutOverChans150.h5",
        "key": "meanH",
        "name": "AS_high",
        "description": "Envelop intensity (a.u.) in 20-40 Hz band during AS"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/asBoutOverChans150.h5",
        "key": "meanLQS",
        "name": "QS_low",
        "description": "Envelop intensity (a.u.) in 0.1-10 Hz band during QS"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/asBoutOverChans150.h5",
        "key": "meanHQS",
        "name": "QS_high",
        "description": "Envelop intensity (a.u.) in 20-40 Hz band during QS"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/wakeBoutOverChans150.h5",
        "key": "meanL",
        "name": "wake_low",
        "description": "Envelop intensity (a.u.) in 0.1-10 Hz band during wake"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/wakeBoutOverChans150.h5",
        "key": "meanH",
        "name": "wake_high",
        "description": "Envelop intensity (a.u.) in 20-40 Hz band during wake"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/spindleRate.h5",
        "key": "smoothedRate",
        "name": "spindle_rate",
        "description": "Spindle rate (events per min) during QS"
    },
#     {
#         "path": "/bucket/ReiterU/octopus/electrophysiology/bceBoutOverChans.h5",
#         "key": "maxBCE",
#         "name": "BCE-LFP_xcr",
#         "description": "Correlation between BCE and LFP during QS"
#     },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/vleRate.h5",
        "key": "smoothedRate",
        "name": "VLE_rate",
        "description": "VLE rate"
    },
]

In [7]:
probes = merge_data_oct(data, hdf_info, lut)

In [8]:
# remove aBL channels, since the sample size is too low
probes.drop(probes[probes["Region_name"]=="aBL"].index, inplace=True)

In [9]:
display(probes)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region,Region_name,Region_color,AS_low,AS_high,QS_low,QS_high,wake_low,wake_high,spindle_rate,VLE_rate,probe
0,0,188.530,182.238,151.016,1787.73,2235.010,1369.73,7,Subfr,"[251, 202, 0]",8.031801,5.352962,5.310945,4.910202,10.110984,4.588534,0.002035,0.009922,L-J30
1,1,189.106,182.020,151.850,1790.54,2227.030,1374.33,6,Buc,"[244, 128, 26]",8.387841,5.550501,5.648776,5.406944,13.041589,4.845503,0.002358,0.009922,L-J30
2,2,189.691,181.804,152.705,1793.35,2219.050,1378.92,6,Buc,"[244, 128, 26]",8.435032,5.550501,5.743419,5.406944,14.179334,4.863403,0.002367,0.011709,L-J30
3,3,190.277,181.584,153.564,1796.16,2211.070,1383.52,6,Buc,"[244, 128, 26]",8.993790,5.369701,5.644291,4.921538,15.057707,4.723267,0.002367,0.011709,L-J30
4,4,190.866,181.371,154.449,1798.97,2203.080,1388.11,6,Buc,"[244, 128, 26]",8.471179,5.293127,5.703934,4.814057,19.834855,4.694675,0.002619,0.014387,L-J30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,182,242.226,151.987,424.261,1992.10,478.734,2870.82,1,VL,"[128, 174, 128]",10.184767,3.927582,6.648152,3.791308,13.796522,3.965979,0.001230,0.006667,L-A29
1964,183,242.312,151.292,425.283,1991.75,469.050,2873.30,1,VL,"[128, 174, 128]",9.585992,3.805563,6.648152,3.684484,13.507777,3.876493,0.001230,0.005833,L-A29
1965,184,242.384,150.592,426.303,1991.21,459.367,2875.72,1,VL,"[128, 174, 128]",9.398674,4.018981,6.356279,3.936605,15.017045,4.075801,0.001230,0.005833,L-A29
1966,185,242.443,149.871,427.341,1990.49,449.684,2878.09,1,VL,"[128, 174, 128]",8.582735,3.805563,5.984127,3.684484,12.771406,3.875377,0.001230,0.003333,L-A29


### Show probe locations

In [10]:
threeD.render_probes_on_surface(probes, "all", "AS_high", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=10000, surface_coloring_by="region", probe_colormap="magma",
                                surface_opacity=0.25,
                                savename=os.path.join(tmpdir, "probe_locations.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

### AS, high-frequency

In [11]:
threeD.render_probes_on_surface(probes, "all", "AS_high", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=12, surface_coloring_by="intensity", surface_colormap="Reds",
                                savename=os.path.join(tmpdir, "3D_AS_highfreq.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

In [12]:
threeD.render_probes_on_surface(probes, "all", "AS_high", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=12, surface_coloring_by="gray", tube_radius=2,
                                savename=os.path.join(tmpdir, "3D_AS_highfreq_gray.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

### AS, low-frequency

In [13]:
threeD.render_probes_on_surface(probes, "all", "AS_low", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=45, surface_coloring_by="intensity", surface_colormap="Reds",
                                savename=os.path.join(tmpdir, "3D_AS_lowfreq.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

In [14]:
threeD.render_probes_on_surface(probes, "all", "AS_low", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=45, surface_coloring_by="gray", tube_radius=2,
                                savename=os.path.join(tmpdir, "3D_AS_lowfreq_gray.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

### QS, high frequency

In [15]:
threeD.render_probes_on_surface(probes, "all", "QS_high", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=12, surface_coloring_by="intensity", surface_colormap="Reds",
                                savename=os.path.join(tmpdir, "3D_QS_highfreq.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

In [16]:
threeD.render_probes_on_surface(probes, "all", "QS_high", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=12, surface_coloring_by="gray", tube_radius=2,
                                savename=os.path.join(tmpdir, "3D_QS_highfreq_gray.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

### QS, low frequency

In [17]:
threeD.render_probes_on_surface(probes, "all", "QS_low", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=45, surface_coloring_by="intensity", surface_colormap="Reds",
                                savename=os.path.join(tmpdir, "3D_QS_lowfreq.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

In [18]:
threeD.render_probes_on_surface(probes, "all", "QS_low", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=45, surface_coloring_by="gray", tube_radius=2,
                                savename=os.path.join(tmpdir, "3D_QS_lowfreq_gray.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

### Wake, high frequency

In [19]:
threeD.render_probes_on_surface(probes, "all", "wake_high", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=12, surface_coloring_by="intensity", surface_colormap="Reds",
                                savename=os.path.join(tmpdir, "3D_wake_highfreq.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

In [20]:
threeD.render_probes_on_surface(probes, "all", "wake_high", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=12, surface_coloring_by="gray", tube_radius=2,
                                savename=os.path.join(tmpdir, "3D_wake_highfreq_gray.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

### Wake, low frequency

In [21]:
threeD.render_probes_on_surface(probes, "all", "wake_low", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=45, surface_coloring_by="intensity", surface_colormap="Reds",
                                savename=os.path.join(tmpdir, "3D_wake_lowfreq.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

In [22]:
threeD.render_probes_on_surface(probes, "all", "wake_low", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=1, vmax=45, surface_coloring_by="gray", tube_radius=2,
                                savename=os.path.join(tmpdir, "3D_wake_lowfreq_gray.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

### Save results to bucket

In [23]:
resultdir = "/bucket/ReiterU/DBS/analysis/230123_octopus_AS_QS_wake_figures"
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [24]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/230123_neuropixel_3D_plot_remake/3D', 'deigo:/bucket/ReiterU/DBS/analysis/230123_octopus_AS_QS_wake_figures'], returncode=0)